In [1]:
import copy
from fastfusion import Specification
spec = Specification.from_yaml("architecture/four_level.arch.yaml")

spec.estimate_energy_area()
spec.component_energy.to_yaml("ERT.yaml")
spec.component_area.to_yaml("ART.yaml")

2025-05-05 12:07:44 INFO        Loading yaml file architecture/four_level.arch.yaml
2025-05-05 12:07:44 INFO        Loading yaml file architecture/four_level.arch.yaml
2025-05-05 12:07:44 INFO        Found top-key variables in architecture/four_level.arch.yaml
2025-05-05 12:07:44 INFO        Found top-key architecture in architecture/four_level.arch.yaml
2025-05-05 12:07:44 INFO        Found top-key compound_components in architecture/four_level.arch.yaml
2025-05-05 12:07:44 WARNING     Loading configuration file from /root/.config/fastfusion/config.yaml
2025-05-05 12:07:44 INFO        Loading yaml file /root/.config/fastfusion/config.yaml
2025-05-05 12:07:44 INFO        Loading yaml file /root/.config/fastfusion/config.yaml
2025-05-05 12:07:44 INFO        Found top-key version in /root/.config/fastfusion/config.yaml
2025-05-05 12:07:44 INFO        Found top-key environment_variables in /root/.config/fastfusion/config.yaml
2025-05-05 12:07:44 INFO        Found top-key expression_custom

In [2]:
# SIM: Track all aliased tensors. Once we combine two Einsums, if we have an alias
# in our reservations, then subtract one of them.

# Process results: Given a list of (tensor, alias) pairs, if an 

from collections import defaultdict
import importlib
import numpy as np
import fastfusion.visualization.interactive as interactive
import fastfusion.visualization.reservationtree as looptree
from tqdm import tqdm
from IPython.display import display, SVG
importlib.reload(interactive)
importlib.reload(looptree)
import pandas as pd
to_plot = {k: exp.inter_result for k, exp in results.items()}
from fastfusion.visualization.interactive import mapping2svg
interactive.plotly_show(to_plot, "Energy", "Latency", logscales=1, mapping_svg=True)

ModuleNotFoundError: No module named 'fastfusion.tags'

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

speed_to_plot = {k: v.history for k, v in speed.items()}
speed_to_plot = copy.deepcopy(speed_to_plot)
speed_to_plot = {k.split('|')[-1].strip(): v for k, v in speed_to_plot.items()}
speed_to_plot = {k: v for k, v in list(speed_to_plot.items())[::-1]}
fastfusion_min_score = min(v[1] for v in speed_to_plot["Fastfusion"])
fastfusion_n_evaluations = max(x[0] for x in speed_to_plot["Fastfusion"])
# Make sure all scores are >= fastfusion_min_score
for k, v in speed_to_plot.items():
    v = [(x[0], max(x[1], fastfusion_min_score)) for x in v]
    speed_to_plot[k] = v

# Normalize to fastfusion
for k, v in speed_to_plot.items():
    v = [(x[0], x[1] / fastfusion_min_score) for x in v]
    speed_to_plot[k] = v

# fastfusion_n_evaluations = min(x[0] for x in speed_to_plot["Fastfusion"])
# Remove inf values
inf = float("inf")
speed_to_plot = {k: list(filter(lambda x: x[1] != inf, v)) for k, v in speed_to_plot.items()}
max_evaluations = max(max(v2[0] for v2 in v) for v in speed_to_plot.values())
if max_evaluations == fastfusion_n_evaluations:
    max_evaluations *= 2
min_score = min(min(v2[1] for v2 in v) for v in speed_to_plot.values())
for k, v in speed_to_plot.items():
    v.append((max_evaluations, v[-1][1]))
# History is a list of tuples (score, evaluations)
fig, ax = plt.subplots(figsize=(10, 6))
for k, v in speed_to_plot.items():
    v = np.array(v)
    ax.plot(v[:, 0], v[:, 1], label=k)
ax.set_xlabel("Evaluations")
ax.set_ylabel("Energy-Delay Product")
# ax.set_yscale("log")
# ax.set_xlim(left=fastfusion_n_evaluations / 10, right=max_evaluations)
ax.set_ylim(bottom=0, top=min_score * 3)
ax.set_xscale("log")
ax.legend()

In [ ]:
v = next(iter(results.values())).n_mappings_inter.n_mappings

# Create a bar chart
plt.figure(figsize=(10, 6))
plt.bar(v.keys(), v.values(), color='skyblue')

values = list(v.values())
for i, value in enumerate(values):
    if i > 0 and values[i] > values[i-1]:
        print(values[i])

# Set logarithmic scale for Y-axis
plt.yscale('log')

# Add labels and title
plt.ylabel('# of Mappings')

# Rotate X-axis labels vertically
plt.xticks(rotation=90)

# Show the plot
plt.show()